In [8]:
import pandas as pd
import yfinance as yf
from datetime import datetime
from datetime import date,timedelta
import yfinance as yf
import plotly.graph_objects as go
import plotly.express as px
import matplotlib.pyplot as plt
from prophet import Prophet
from prophet.plot import plot_plotly, plot_components_plotly
import seaborn as sns
import pandas_ta as ta
import numpy as np
import json
from tkinter import *
from tkinter import messagebox, ttk, END
from matplotlib import dates

In [9]:
today = date.today()

d1 = today.strftime('%Y-%m-%d')
end_date = d1
d2 = date.today() - timedelta(days=365)
d2 = d2.strftime('%Y-%m-%d')
start_date = d2

In [10]:
from curl_cffi import requests
session = requests.Session(impersonate="chrome")
ticker = input('Write ticker: ')
stock_data = yf.download(ticker, start = start_date, end = end_date, progress=False, auto_adjust=False, session=session)
if isinstance(stock_data.columns, pd.MultiIndex):
    stock_data.columns = stock_data.columns.droplevel(1)
stock_data

Price,Adj Close,Close,High,Low,Open,Volume
Date,,,,,,
2025-02-11,2.414216,2.414216,2.525392,2.376361,2.422748,5368991525
2025-02-12,2.472835,2.472835,2.490575,2.349436,2.414219,5462469809
2025-02-13,2.562515,2.562515,2.596130,2.424654,2.472843,5041669574
2025-02-14,2.740196,2.740196,2.828271,2.535260,2.562502,8841598608
2025-02-15,2.761578,2.761578,2.827311,2.721855,2.740087,4633704309
...,...,...,...,...,...,...
2026-02-06,1.471523,1.471523,1.538908,1.133460,1.213506,11836014443
2026-02-07,1.424204,1.424204,1.482395,1.388035,1.471422,5355780912
2026-02-08,1.432185,1.432185,1.465540,1.413845,1.424202,2764477043


In [11]:
#RSI, Garman, ATR, MACD
stock_data['RSI'] = ta.rsi(close = stock_data["Adj Close"], length=14)

stock_data['Garman'] = ((np.log(stock_data['High'])- np.log(stock_data['Low']))**2)/2-(2*np.log(2)-1)*((np.log(stock_data['Adj Close']) -np.log(stock_data['Open']))**2)

def compute_atr(stock_data):
    atr = ta.atr(high=stock_data['High'], low=stock_data['Low'], close=stock_data['Close'], length=14)
    return atr.sub(atr.mean()).div(atr.std())
stock_data['ATR'] = compute_atr(stock_data)

def compute_macd(stock_data, short_period=12, long_period=26, signal_period=9):
    # Compute MACD, Signal Line, and Histogram using pandas_ta
    macd = ta.macd(
        close=stock_data['Close'], 
        fast=short_period, 
        slow=long_period, 
        signal=signal_period
    )
    
    # Assign the results to new columns in the DataFrame
    stock_data['MACD'] = macd['MACD_12_26_9']  # Default naming format
    stock_data['Signal'] = macd['MACDs_12_26_9']
    stock_data['MACD_Histogram'] = macd['MACDh_12_26_9']
    
    return stock_data

# Apply the function to compute MACD
stock_data = compute_macd(stock_data)

today = date.today() 
two_weeks = today - timedelta(days=14)

#Last 14 days
technical_analysis = stock_data.iloc[-14:]
technical_analysis

Price,Adj Close,Close,High,Low,Open,Volume,RSI,Garman,ATR,MACD,Signal,MACD_Histogram
Date,,,,,,,,,,,,
2026-01-28,1.908726,1.908726,1.938175,1.893775,1.913928,2302935086,43.510402,0.000266,-1.394440,-0.038440,-0.024497,-0.013943
2026-01-29,1.805065,1.805065,1.909080,1.777273,1.908756,4595244677,36.425414,0.001354,-1.299911,-0.046180,-0.028833,-0.017346
2026-01-30,1.732426,1.732426,1.809327,1.713048,1.805088,5054515535,32.439243,0.000843,-1.276680,-0.057512,-0.034569,-0.022943
2026-01-31,1.645471,1.645471,1.749177,1.544586,1.732426,6222981594,28.428577,0.006712,-1.058329,-0.072672,-0.042190,-0.030482
2026-02-01,1.591342,1.591342,1.673348,1.560783,1.645466,4092694363,26.252676,0.001993,-1.022767,-0.088039,-0.051360,-0.036679
2026-02-02,1.620547,1.620547,1.653190,1.532506,1.591397,5046409178,29.392795,0.002746,-0.974994,-0.096746,-0.060437,-0.036309
2026-02-03,1.574199,1.574199,1.625398,1.528746,1.620558,4089022562,27.398948,0.001554,-0.974294,-0.106162,-0.069582,-0.036580
2026-02-04,1.510476,1.510476,1.608769,1.497360,1.574226,4190815541,24.898212,0.001915,-0.946834,-0.117413,-0.079148,-0.038265
2026-02-05,1.213521,1.213521,1.510476,1.150371,1.510476,11340956282,17.076371,0.018575,-0.469508,-0.148579,-0.093034,-0.055544


In [12]:
import plotly.graph_objects as go

# ----------------------------
# Indicators
# ----------------------------
stock_data["M10"] = stock_data["Close"].rolling(window=10).mean()
stock_data["M20"] = stock_data["Close"].rolling(window=20).mean()

stock_data["M40"] = stock_data["Close"].rolling(window=40).mean()
stock_data["M80"] = stock_data["Close"].rolling(window=80).mean()

stock_data["SD"] = stock_data["Close"].rolling(window=20).std()
stock_data["UB"] = stock_data["M20"] + 2 * stock_data["SD"]
stock_data["LB"] = stock_data["M20"] - 2 * stock_data["SD"]


fig = go.Figure()

# 0 Close (line)
fig.add_trace(go.Scatter(
    x=stock_data.index, y=stock_data["Close"],
    mode="lines", name="Close"
))

# 1 Candles
fig.add_trace(go.Candlestick(
    x=stock_data.index,
    open=stock_data["Open"],
    high=stock_data["High"],
    low=stock_data["Low"],
    close=stock_data["Close"],
    name="Candles"
))

# 2-3 MA 10/20
fig.add_trace(go.Scatter(x=stock_data.index, y=stock_data["M10"], mode="lines", name="M10"))
fig.add_trace(go.Scatter(x=stock_data.index, y=stock_data["M20"], mode="lines", name="M20"))

# 4-5 MA 40/80
fig.add_trace(go.Scatter(x=stock_data.index, y=stock_data["M40"], mode="lines", name="M40"))
fig.add_trace(go.Scatter(x=stock_data.index, y=stock_data["M80"], mode="lines", name="M80"))

# 6-8 Bollinger Bands
fig.add_trace(go.Scatter(
    x=stock_data.index, y=stock_data["UB"],
    mode="lines", name="Upper BB"
))
fig.add_trace(go.Scatter(
    x=stock_data.index, y=stock_data["LB"],
    mode="lines", name="Lower BB",
    fill="tonexty"  # fill between UB (previous trace) and LB
))
fig.add_trace(go.Scatter(
    x=stock_data.index, y=stock_data["M20"],
    mode="lines", name="Middle BB (M20)"
))

# ----------------------------
# Visibility presets (length MUST be 9)
# ----------------------------
PRICE_CANDLES = [False, True,  False, False, False, False, False, False, False]
PRICE_LINE    = [True,  False, False, False, False, False, False, False, False]

MA_10_20      = [True,  False, True,  True,  False, False, False, False, False]
MA_40_80      = [True,  False, False, False, True,  True,  False, False, False]

BB_VIEW       = [True,  False, False, True,  False, False, True,  True,  True]
# (BB_VIEW includes Close + M20 + UB/LB + Middle; you can tweak if you want)

# Default view
for t in fig.data:
    t.visible = False
fig.data[1].visible = True  # default = candles

# ----------------------------
# Layout + Buttons (positioned to avoid title conflict)
# ----------------------------
fig.update_layout(
    title=dict(text=f"{ticker} Price", x=0.01, y=0.97),  # keep title slightly lower
    xaxis_title="Date",
    yaxis_title="Price",
    plot_bgcolor="lightgrey",
    xaxis_rangeslider_visible=False,
    width=1050,
    height=650,
    margin=dict(t=110, r=30, l=60, b=50),  # extra top margin for buttons + title
    updatemenus=[
        dict(
            type="buttons",
            direction="right",
            x=1.0,
            y=1.16,
            xanchor="right",
            yanchor="top",
            pad={"r": 10, "t": 10},
            buttons=[
                dict(
                    label="Candles",
                    method="update",
                    args=[{"visible": PRICE_CANDLES},
                          {"title": f"{ticker} Price (Candles)", "yaxis": {"title": "Price"}}],
                ),
                dict(
                    label="Line",
                    method="update",
                    args=[{"visible": PRICE_LINE},
                          {"title": f"{ticker} Price (Line)", "yaxis": {"title": "Price"}}],
                ),
                dict(
                    label="MA 10/20",
                    method="update",
                    args=[{"visible": MA_10_20},
                          {"title": f"{ticker} Moving Averages (10/20)", "yaxis": {"title": "Price"}}],
                ),
                dict(
                    label="MA 40/80",
                    method="update",
                    args=[{"visible": MA_40_80},
                          {"title": f"{ticker} Moving Averages (40/80)", "yaxis": {"title": "Price"}}],
                ),
                dict(
                    label="Bollinger",
                    method="update",
                    args=[{"visible": BB_VIEW},
                          {"title": f"{ticker} Bollinger Bands (20, 2σ)", "yaxis": {"title": "Price"}}],
                ),
            ],
        )
    ],
)

fig.show()


Sharpe and Sortino Ratio

In [13]:
stock_data = stock_data.copy()
stock_data["Daily Returns"] = stock_data["Adj Close"].pct_change()
stock_data.dropna(inplace=True)

def compute_sharpe_ratio(data, risk_free_rate=0, periods=365):
    """
    Computes daily and annualized Sharpe Ratio
    """
    returns = data["Daily Returns"]
    excess_returns = returns - risk_free_rate / periods

    daily_sharpe = excess_returns.mean() / returns.std()
    annual_sharpe = daily_sharpe * np.sqrt(periods)

    return daily_sharpe, annual_sharpe

def compute_sortino_ratio(data, threshold=0, risk_free_rate=0, periods=365):
    """
    Computes annualized Sortino Ratio
    """
    returns = data["Daily Returns"]
    downside_returns = returns[returns < threshold]

    downside_std = downside_returns.std()
    excess_return = returns.mean() - risk_free_rate / periods

    sortino = (excess_return / downside_std) * np.sqrt(periods)
    return sortino


daily_sharpe, annual_sharpe = compute_sharpe_ratio(stock_data)
sortino = compute_sortino_ratio(stock_data)

print(f"Daily Sharpe Ratio for {ticker}: {daily_sharpe:.4f}")
print(f"Annual Sharpe Ratio for {ticker}: {annual_sharpe:.4f}")
print(f"Sortino Ratio for {ticker}: {sortino:.4f}")


Daily Sharpe Ratio for XRP-USD: -0.0208
Annual Sharpe Ratio for XRP-USD: -0.3969
Sortino Ratio for XRP-USD: -0.5955


In [14]:
prices = pd.concat([xrp, btc, eth], axis=1)
prices.columns = ["XRP", "BTC", "ETH"]
prices = prices.dropna()

# daily log returns
log_returns = np.log(prices / prices.shift(1)).dropna()

def monte_carlo_paths(price_series, mu, sigma, days=365, n_sims=5000, seed=42):
    """
    Geometric Brownian Motion (GBM) simulation using log returns
    """
    rng = np.random.default_rng(seed)
    s0 = float(price_series.iloc[-1])

    # random daily shocks
    z = rng.standard_normal((days, n_sims))

    # GBM in log space: S(t) = S0 * exp( (mu - 0.5*sigma^2)*t + sigma*W )
    drift = (mu - 0.5 * sigma**2)
    daily_log_steps = drift + sigma * z
    log_paths = np.cumsum(daily_log_steps, axis=0)
    paths = s0 * np.exp(log_paths)

    # add S0 as day 0
    paths = np.vstack([np.full((1, n_sims), s0), paths])
    return paths  # shape: (days+1, n_sims)

def summarize_paths(paths):
    terminal = paths[-1, :]
    s0 = paths[0, 0]
    summary = {
        "Start": s0,
        "Mean End": terminal.mean(),
        "Median End": np.median(terminal),
        "P05 End": np.percentile(terminal, 5),
        "P95 End": np.percentile(terminal, 95),
        "Prob End > Start": (terminal > s0).mean()
    }
    return summary, terminal

DAYS = 365
N_SIMS = 50000

results = {}
terminals = {}

for coin in ["XRP", "BTC", "ETH"]:
    mu = log_returns[coin].mean()
    sigma = log_returns[coin].std()

    paths = monte_carlo_paths(prices[coin], mu, sigma, days=DAYS, n_sims=N_SIMS, seed=42)
    summary, terminal = summarize_paths(paths)

    results[coin] = summary
    terminals[coin] = terminal

# Print results
for coin, s in results.items():
    print(f"\n--- {coin} Monte Carlo ({DAYS} days, {N_SIMS} sims) ---")
    print(f"Start:            {s['Start']:.6f}")
    print(f"Mean End:         {s['Mean End']:.6f}")
    print(f"Median End:       {s['Median End']:.6f}")
    print(f"5th % End:        {s['P05 End']:.6f}")
    print(f"95th % End:       {s['P95 End']:.6f}")
    print(f"Prob End > Start: {s['Prob End > Start']*100:.2f}%")


coin_list = ["XRP", "BTC", "ETH"]

PLOT_SIMS = 200
plot_paths = {}

for coin in coin_list:
    mu = log_returns[coin].mean()
    sigma = log_returns[coin].std()
    plot_paths[coin] = monte_carlo_paths(prices[coin], mu, sigma, days=DAYS, n_sims=PLOT_SIMS, seed=7)

fig = go.Figure()

trace_groups = {}

for i, coin in enumerate(coin_list):
    paths = plot_paths[coin]
    trace_idxs = []

    # spaghetti paths
    for s in range(paths.shape[1]):
        fig.add_trace(go.Scatter(
            x=np.arange(paths.shape[0]),
            y=paths[:, s],
            mode="lines",
            name=f"{coin} sim",
            line=dict(width=1),
            opacity=0.15,
            showlegend=False,
            visible=(i == 0)
        ))
        trace_idxs.append(len(fig.data) - 1)

    # mean path
    fig.add_trace(go.Scatter(
        x=np.arange(paths.shape[0]),
        y=paths.mean(axis=1),
        mode="lines",
        name=f"{coin} mean",
        line=dict(width=3),
        visible=(i == 0)
    ))
    trace_idxs.append(len(fig.data) - 1)

    trace_groups[coin] = trace_idxs

buttons = []
for coin in coin_list:
    visible = [False] * len(fig.data)
    for idx in trace_groups[coin]:
        visible[idx] = True

    buttons.append(dict(
        label=coin,
        method="update",
        args=[
            {"visible": visible},
            {"title": f"{coin} Monte Carlo Paths ({DAYS} days, {PLOT_SIMS} sims shown)",
             "xaxis": {"title": "Day"},
             "yaxis": {"title": "Simulated Price"}}
        ],
    ))

fig.update_layout(
    title=f"XRP Monte Carlo Paths ({DAYS} days, {PLOT_SIMS} sims shown)",
    width=1050,
    height=600,
    plot_bgcolor="lightgrey",
    updatemenus=[dict(
        type="buttons",
        direction="right",
        x=1.0,
        y=1.15,
        xanchor="right",
        yanchor="top",
        pad={"r": 10, "t": 10},
        buttons=buttons
    )],
    margin=dict(t=110, r=30, l=60, b=50)
)

fig.show()

fig2 = go.Figure()

hist_groups = {}
for i, coin in enumerate(coin_list):
    terminal = terminals[coin]
    fig2.add_trace(go.Histogram(
        x=terminal,
        nbinsx=60,
        name=f"{coin} terminal",
        opacity=0.75,
        visible=(i == 0)
    ))
    hist_groups[coin] = len(fig2.data) - 1

buttons2 = []
for coin in coin_list:
    visible = [False] * len(fig2.data)
    visible[hist_groups[coin]] = True

    s = results[coin]
    subtitle = (f"Start={s['Start']:.6f} | Mean={s['Mean End']:.6f} | "
                f"P05={s['P05 End']:.6f} | P95={s['P95 End']:.6f} | "
                f"P(End>Start)={s['Prob End > Start']*100:.2f}%")

    buttons2.append(dict(
        label=coin,
        method="update",
        args=[
            {"visible": visible},
            {"title": f"{coin} Terminal Price Distribution<br><sup>{subtitle}</sup>",
             "xaxis": {"title": "Terminal Price"},
             "yaxis": {"title": "Count"}}
        ],
    ))

fig2.update_layout(
    title="XRP Terminal Price Distribution",
    width=1050,
    height=600,
    plot_bgcolor="lightgrey",
    updatemenus=[dict(
        type="buttons",
        direction="right",
        x=1.0,
        y=1.15,
        xanchor="right",
        yanchor="top",
        pad={"r": 10, "t": 10},
        buttons=buttons2
    )],
    margin=dict(t=110, r=30, l=60, b=50)
)

fig2.show()


NameError: name 'xrp' is not defined